# AES Plaintext Recovery (ResNet)
In this experiment, the network tries to guess the plaintext from the ciphertext, helped with ascii per-byte correction. 

## Imports

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import SimplifiedAESDatasetCiphertextPlaintext
from pipeline import *

## Importing the dataset

In [37]:
data = SimplifiedAESDatasetCiphertextPlaintext('small')

train_labels, train_samples, test_labels, test_samples = data.get_data()

In [38]:
train_labels = train_labels[:500]
train_samples = train_samples[:500]

In [39]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (500, 16)
===== Label Shape: (16,)
===== Training Samples Shape: (500, 16)
===== Sample Shape: (16,)
===== Testing Labels Shape: (649947, 16)
===== Testing Samples Shape: (649947, 16)


## Creating the model

In [40]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [41]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*128
print(units)

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=10000,
    decay_rate=0.01)
optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 1000
batch_size = 5000

2048


### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [42]:

inputs = Input(shape=input_shape)
net = inputs

for _ in range(5):
    net = residual_block(net, units)
    
net = Dense(units=dim, activation='softmax')(net)

neural_network = Model(inputs, net)

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 batch_normalization_46 (BatchN  (None, 16)          64          ['input_6[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 dense_51 (Dense)               (None, 2048)         34816       ['batch_normalization_46[0][0]'] 
                                                                                                  
 batch_normalization_47 (BatchN  (None, 2048)        8192        ['dense_51[0][0]']         

### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [43]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.3941 - accuracy: 0.0400 - binary_accuracy: 0.5683 - val_loss: 0.3840 - val_accuracy: 0.0000e+00 - val_binary_accuracy: 0.5638
Epoch 2/1000
1/1 [==============================] - 1s 513ms/step - loss: 0.3593 - accuracy: 0.1956 - binary_accuracy: 0.6104 - val_loss: 0.3825 - val_accuracy: 0.0000e+00 - val_binary_accuracy: 0.5638
Epoch 3/1000
1/1 [==============================] - 1s 517ms/step - loss: 0.3574 - accuracy: 0.1644 - binary_accuracy: 0.6251 - val_loss: 0.3809 - val_accuracy: 0.0600 - val_binary_accuracy: 0.5638
Epoch 4/1000
1/1 [==============================] - 1s 512ms/step - loss: 0.3606 - accuracy: 0.1067 - binary_accuracy: 0.6306 - val_loss: 0.3782 - val_accuracy: 0.0800 - val_binary_accuracy: 0.5638
Epoch 5/1000
1/1 [==============================] - 1s 517ms/step - loss: 0.3611 - accuracy: 0.1000 - binary_accuracy: 0.6301 - val_loss: 0.3759 - val_accuracy: 0.0400 - val_binary_accuracy: 0.5713
Epoch

In [44]:
predict_sample(neural_network, test_samples[48])

1/1 [==============================] - 0s 262ms/step


array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [45]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

130/130 [==============================] - 196s 2s/step - loss: 0.3899 - accuracy: 0.1686 - binary_accuracy: 0.6101
Test loss: 0.3898623585700989
Test accuracy: 0.16863836348056793


In [46]:
results, pred = test_model(neural_network, test_samples, test_labels, batch_size, ascii_correction=True)

130/130 [==============================] - 191s 1s/step


In [47]:
print("Correct bytes: " + str(results["correct_bytes"]))
print("Byte accuracy: " + str(results["byte_accuracy"]))

print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))

Correct bytes: 0
Byte accuracy: 0.0
Correct predictions: 0
Accuracy: 0.0
